# Retinal Disease Classification Utilizing Fundus Images of the Eye

## By: Jalen Wu, Yechan Na, Jonathan Zhang

__Project Description:__

The goal of this project is to develop a machine learning model capable of detecting retinal diseases by analyzing images of the eye. Using computer vision and deep learning techniques, the model assists in early detection and diagnosis of retinal disease(s).

__Applications and Impact:__

This project could be used for clinical screening to help ophthalmologists identify diseases and improve efficiency on identifying these diseases. This automated detection system has the potential to make medical imaging diagnostics more accessible.

__What we hope to achieve:__

We hope to build a model that takes in images of the eye and accurately predicts whether an individual’s eyes are healthy or showing signs of disease. To quantify the effectiveness of our model, we will be measuring metrics such as F1 score, precision, recall, loss, and accuracy of our models and graphing them as a function of how many epochs we run on our training data (all of these scores should increase in subsequent epochs).

- __Dataset__: https://www.kaggle.com/datasets/andrewmvd/retinal-disease-classification/data
- __References__: 
    - https://www.mdpi.com/2306-5729/6/2/14
    - https://jamanetwork.com/journals/jama/fullarticle/2588763
    

In [1]:
# imports
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms
import torch.nn.functional as F
import numpy as np
from matplotlib import pyplot as plt

x = torch.rand([5,5])
print(x)

tensor([[0.2047, 0.4721, 0.7078, 0.2602, 0.2930],
        [0.0599, 0.5772, 0.1006, 0.6206, 0.2996],
        [0.7264, 0.4252, 0.6360, 0.2271, 0.5815],
        [0.0450, 0.7709, 0.2046, 0.9319, 0.0802],
        [0.7952, 0.2234, 0.2588, 0.3427, 0.9440]])


In [2]:
# Import Dataset

import kagglehub

# Download latest version
path = kagglehub.dataset_download("andrewmvd/retinal-disease-classification")

print("Path to dataset files:", path)

c:\Users\Jalen\anaconda3\envs\env_pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 7.43G/7.43G [01:56<00:00, 68.4MB/s]

Extracting files...


Path to dataset files: C:\Users\Jalen\.cache\kagglehub\datasets\andrewmvd\retinal-disease-classification\versions\1


In [ ]:
# X = 1424 x 2144 x 3 : h x w x colors
# y = label

transform = transforms.Compose([
    transforms.Resize((1424, 2144)), # Standardize image dimensions to 1444 x 2144
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

In [ ]:
# # Download training data from open datasets.
# training_data = datasets.CIFAR10(
#     root="data",
#     train=True,
#     download=True,
#     transform=ToTensor(),
# )

# Path to the dataset folder
train_dir = path + '/Training_Set/Training_Set/Training'
test_dir = path + '/Test_Set/Test_Set/Test'

# Load the datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)


# # Download test data from open datasets.
# test_data = datasets.CIFAR10(
#     root="data",
#     train=False,
#     download=True,
#     transform=ToTensor(),
# )

# label_names = datasets.CIFAR10(root="data", download=False).classes
# print(label_names)

# Analysis
- Plot loss/accuracy (y) with number of epochs ran (x)
- Recall/Precision/F1 score 